In [2]:
import pandas as pd

import csv_utils

In [98]:
df = pd.read_csv('./../data/BBC/bbc_data.csv', sep=';', index_col=0)
#df = pd.read_csv('./../data/BBC/bbc_images.csv', sep=';', index_col=0)

In [99]:
df

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247,music
1,1,Port,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037,music
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,10 Feb 2022,611,music
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ceiliúradh na Féile Pádraig",John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,False,17 Mar 2017,3538,music
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Songs of Praise, Edinburgh",Claire McCollum explores Edinburgh and shares ...,Claire McCollum goes on a Christian heritage t...,Claire McCollum explores Edinburgh and discove...,Music,bbc_one,False,1:15pm 20 Feb 2022,2044,music
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3798,3798,Murder Trial: The Disappearance of Margaret Fl...,The trial continues as Cairney and Jones stand...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Murder Trial: The Disappeara...",The trial continues as Cairney and Jones stand...,Second part of the documentary about the trial...,As the murder trial that gripped a nation cont...,Documentary,bbc_two,Contains some upsetting scenes.,10pm 8 Jan 2020,3540,documentaries
3799,3799,"Paul Merson: Football, Gambling and Me",Footballer Paul Merson sets out to understand ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Paul Merson: Football, Gambl...",Footballer Paul Merson sets out to understand ...,Former footballer Paul Merson sets out to unde...,"Over the past 35 years, former Arsenal and Eng...",Sport,bbc_one,Contains some strong language.,9pm 11 Oct 2021,3480,documentaries
3800,3800,Avicii: True Stories,Avicii: True Stories is Tim Bergling’s own sto...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Avicii: True Stories",Avicii: True Stories is Tim Bergling’s own sto...,Avicii: True Stories is Tim Bergling’s own sto...,"In 2006, Tim Bergling was just a regular teena...",Documentary,bbc_three,Contains strong language.,20 Apr 2019,5766,documentaries
3801,3801,Heaven Made,"On the Isle of Wight, Brother Matthew, a monk ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Heaven Made, Series 1: Episo...","On the Isle of Wight, Brother Matthew, a monk ...","At Quarr Abbey on the Isle of Wight, Brother M...",The Benedictine nuns of Kylemore Abbey on the ...,Documentary,bbc_one,False,10:30am 13 Feb 2022,3244,documentaries


In [126]:
# columns = ['index', 'title', 'description', 'image', 'keywords', 'syno_s', 'syno_m', 'syno_l', 'category', 'channel', 'language', 'release_date', 'duration_sec', 'topic']
#
# df.columns = columns

## Image Processing

In [127]:
s = '192x108'
m = '288x162'
l = '352x198'
def generate_img(df_inp, size):
    return [image.replace('{recipe}', size) for image in df_inp['image']]
# df['image_l'] = generate_img(df, l)

In [152]:
df_img = df.loc[:, ['index', 'image', 'image_s', 'image_m', 'image_l']]
# df_img.to_csv('../data/BBC/bbc_images.csv', sep=';')

## Synopses clearing

In [134]:
len(df[df['syno_s'].isna()])

0

In [145]:
len(df[df['syno_m'].isna()])

0

In [144]:
len(df[df['syno_l'].isna()])

0

In [137]:
def fill_column(df_input, origin_col, gap_filler_col):
    for index, df_entry in df_input.iterrows():
        # checks if the value is the same, if not it is a NaN
        if not df_entry[origin_col] == df_entry[origin_col]:
            df_input.loc[index, [origin_col]] = df_entry[gap_filler_col]

In [138]:
# Substitute all nan syno_m with the syno_s values
# fill_column(df, origin_col='syno_m', gap_filler_col='syno_s')
# Substitute all nan syno_l with the syno_m values
# fill_column(df, origin_col='syno_l', gap_filler_col='syno_m')

In [18]:
# df.to_csv('../data/BBC/bbc_data.csv', sep=';')

## Remove duplicates

In [100]:
df[df['title'] == 'A Life in Ten Pictures']

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic
1956,1956,A Life in Ten Pictures,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Amy Winehouse’s image is known around the worl...,Amy Winehouse’s image is known around the worl...,History,bbc_two,Contains strong language.,9pm 11 Sep 2021,3550,signed
2836,2836,A Life in Ten Pictures,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Amy Winehouse’s image is known around the worl...,Amy Winehouse’s image is known around the worl...,History,bbc_two,Contains strong language.,9pm 11 Sep 2021,3550,history
3486,3486,A Life in Ten Pictures,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Amy Winehouse’s image is known around the worl...,Amy Winehouse’s image is known around the worl...,History,bbc_two,Contains strong language.,9pm 11 Sep 2021,3550,documentaries


In [102]:
grouped_by_title = df.groupby('title')
def remove_duplicates_by_title(grouped_by_title):
    for name, group in grouped_by_title:
        if len(group) == 1:
             continue

        indeces = []
        topics = []

        for i, value in group.iterrows():
            test = value['title']
            topics.append(value['topic'])
            indeces.append(value['index'])

        df.loc[indeces[0], 'topic'] = ', '.join(topics)
        df = df.drop(labels=indeces[1:], axis=0)

In [103]:
df[df['title'] == 'A Life in Ten Pictures']

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic
1956,1956,A Life in Ten Pictures,"Ten pictures, from iconic shots to private sna...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, A Life in Ten Pictures, Seri...","Ten pictures, from iconic shots to private sna...",Amy Winehouse’s image is known around the worl...,Amy Winehouse’s image is known around the worl...,History,bbc_two,Contains strong language.,9pm 11 Sep 2021,3550,"signed, history, documentaries"


In [111]:
#df['topic'] = df['topic'].apply(lambda x: x.split(', '))

In [113]:
df

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247,"[music, from-the-archives]"
1,1,Port,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037,[music]
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,10 Feb 2022,611,"[music, entertainment]"
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ceiliúradh na Féile Pádraig",John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,False,17 Mar 2017,3538,[music]
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Songs of Praise, Edinburgh",Claire McCollum explores Edinburgh and shares ...,Claire McCollum goes on a Christian heritage t...,Claire McCollum explores Edinburgh and discove...,Music,bbc_one,False,1:15pm 20 Feb 2022,2044,"[music, lifestyle]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777,3777,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Models: Street to Catwalk, S...",Kyra’s late arrival puts her modelling career ...,Kyra jeopardises her modelling career by turni...,How will Kyra cope after a train malfunction m...,Documentary,bbc_three,Contains some strong language and some upsetti...,8 Aug 2021,1843,[documentaries]
3789,3789,Death On The I-95,Returning to the US to find out what happened ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Death On The I-95",Returning to the US to find out what happened ...,Following a report on the growing problem of o...,Drug overdose is now the main cause of death f...,Documentary,bbc_three,Contains some strong language and upsetting sc...,24 Oct 2018,3426,[documentaries]
3793,3793,Do Black Lives Still Matter?,"Grime artist Saskilla asks, can the music indu...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Do Black Lives Still Matter?...","Grime artist Saskilla asks, can the music indu...",The music industry pledged millions in support...,The music industry reacted with a surge of out...,Documentary,bbc_three,False,31 May 2021,1524,[documentaries]
3797,3797,Ar An Sliabh,Paul packs up to return to his native Armagh a...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ar An Sliabh, Series 1: Epis...",Paul packs up to return to his native Armagh a...,Paul packs up to return to his native Armagh a...,"I Sliabh Eachtaí, tá Cérill ag déanamh réidh d...",Documentary,bbc_webonly,False,22 Nov 2021,1740,[documentaries]


In [114]:
df.to_csv('../data/BBC/bbc_data.csv', sep=';')

## Genre and Keyword extraction

In [105]:
categories = set(df['category'].tolist())
categories

{'Arts',
 'Beauty',
 'CBBC',
 'CBeebies',
 'Comedy',
 'Comedy Drama',
 'Crime Drama',
 'Current Affairs',
 'Dating',
 'Documentary',
 'Documentary Film',
 'Drama',
 'Entertainment',
 'Fantasy',
 'Film',
 'Film - Action',
 'Film - Adventure',
 'Film - Biopic',
 'Film - Comedy',
 'Film - Crime',
 'Film - Drama',
 'Film - Family',
 'Film - Horror',
 'Film - Musical',
 'Film - Period',
 'Film - Rom-Com',
 'Film - Romance',
 'Film - Sci-Fi',
 'Film - Thriller',
 'Film - War',
 'Film - Western',
 'Food',
 'History',
 'Lifestyle',
 'Music',
 'Nature',
 'News',
 'Period Drama',
 'Reality',
 'Rom-com',
 'Sci-Fi',
 'Science',
 'Science & Nature',
 'Sport',
 nan}

In [106]:
df[df['category'] == 'Film - Adventure']

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic
515,515,Ivanhoe,Ivanhoe fights for the honour of the kidnapped...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ivanhoe",Ivanhoe fights for the honour of the kidnapped...,Swashbuckling spectacular based on the novel b...,Swashbuckling spectacular based on the novel b...,Film - Adventure,bbc_two,False,1952,6136,"films, drama-and-soaps"
538,538,Lara Croft: Tomb Raider,The heroine races against time to find a relic...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Lara Croft: Tomb Raider",The heroine races against time to find a relic...,Action tale adapted from the popular video gam...,Action tale adapted from the popular video gam...,Film - Adventure,bbc_three,False,2001,5614,"films, drama-and-soaps"
546,546,El Cid,The epic story of the 11th-century Spanish her...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, El Cid",The epic story of the 11th-century Spanish her...,Epic story of the Spanish hero (Charlton Hesto...,The epic story of the 11th-century Spanish her...,Film - Adventure,bbc_two,False,1961,10297,"films, drama-and-soaps"
568,568,The Spanish Main,Swashbuckling adventure in which a crew of ens...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, The Spanish Main",Swashbuckling adventure in which a crew of ens...,Swashbuckling adventure in which a Spanish gov...,Early RKO Technicolor swashbuckling adventure....,Film - Adventure,bbc_two,False,1945,5741,"films, drama-and-soaps"
576,576,Lara Croft Tomb Raider: The Cradle of Life,Archaeologist Lara Croft faces a race against ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Lara Croft Tomb Raider: The ...",Archaeologist Lara Croft faces a race against ...,Globetrotting action sequel in which archaeolo...,Globetrotting action sequel in which intrepid ...,Film - Adventure,bbc_three,False,2003,6320,"films, drama-and-soaps"
590,590,Blackbeard the Pirate,Swashbuckling adventure set on the high seas i...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Blackbeard the Pirate",Swashbuckling adventure set on the high seas i...,Swashbuckler set on the high seas in the 17th ...,Swashbuckling adventure set on the high seas i...,Film - Adventure,bbc_two,False,1952,5913,"films, drama-and-soaps"
